In [45]:
from datasets import load_dataset,concatenate_datasets, Dataset,DatasetDict
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch

In [46]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
# def collate(examples):
#     # Convert to Tensors and build dataset

#     input_ids_bert = pad_sequence([torch.tensor(f['bert_token'], dtype=torch.long) for f in examples],
#                                   batch_first=True, padding_value=bert_pad_token)
#     input_ids_gpt = pad_sequence([torch.tensor(f['gpt2_token'], dtype=torch.long) for f in examples],
#                                  batch_first=True, padding_value=gpt2_pad_token)
#     try:
#         token_lengths = torch.tensor([[len(f['bert_token']), len(f['gpt2_token'])] for f in examples],
#                                      dtype=torch.long)
#     except:
#         token_lengths = torch.zeros((len(examples), 1091))
#         for i in range(len(examples)):
#             token_lengths[i, len(examples[i]['gpt2_token'])] = 1
#     return (input_ids_bert, input_ids_gpt, token_lengths)

In [48]:
train_eval_dataset =load_dataset("guangyil/yelp_short_v2")
eval_dataloader =  DataLoader(train_eval_dataset['test'], num_workers=0, collate_fn=collate,batch_size=64)

In [43]:
from models.my_transformers import *
from models.models import VAE
from train.reconstruction import *

In [51]:
encoder_model_class = MODEL_CLASS['BertForLatentConnectorAVG']
tokenizer_encoder = AutoTokenizer.from_pretrained("prajjwal1/bert-small")
latent_size = 64
model_encoder = encoder_model_class.from_pretrained("prajjwal1/bert-small", latent_size=latent_size,
                                                    pad_id=tokenizer_encoder.pad_token_id,local_files_only=False)


decoder_model_class = MODEL_CLASS['GPT2ForLatentConnectorNew']
tokenizer_decoder = AutoTokenizer.from_pretrained("gpt2-xl")
model_decoder = decoder_model_class.from_pretrained("gpt2-xl", latent_size=latent_size,
                                                        latent_as_gpt_emb=True,
                                                        latent_as_gpt_memory=True,local_files_only=False)
decoder_n_layer = model_decoder.transformer.config.n_layer
model_decoder.transformer.change_order()

special_tokens_dict = {'pad_token': '<PAD>', 'bos_token': '<BOS>', 'eos_token': '<EOS>', }
num_added_toks = tokenizer_decoder.add_special_tokens(special_tokens_dict)
model_decoder.resize_token_embeddings(len(tokenizer_decoder))
bert_pad_token = tokenizer_encoder.pad_token_id
gpt2_pad_token = tokenizer_decoder.pad_token_id

Some weights of BertForLatentConnectorAVG were not initialized from the model checkpoint at prajjwal1/bert-small and are newly initialized: ['bert.linear.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPT2ForLatentConnectorNew were not initialized from the model checkpoint at gpt2-xl and are newly initialized: ['h.48.ln_2.bias', 'lm_head.bias', 'h.48.mlp.c_fc.bias', 'linear.weight', 'h.48.attn.c_attn.bias', 'h.48.ln_2.weight', 'h.48.mlp.c_fc.weight', 'linear_emb.weight', 'h.48.attn.c_proj.weight', 'h.48.attn.c_attn.weight', 'h.48.mlp.c_proj.weight', 'h.48.mlp.c_proj.bias', 'h.48.ln_1.weight', 'h.48.attn.c_proj.bias', 'h.48.ln_1.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [54]:
output_dir = "test"
model_vae = VAE(model_encoder, model_decoder, tokenizer_encoder, tokenizer_decoder, latent_size, output_dir)

In [55]:
import torch.nn.init as init
def weights_init_rondom(model):
    model = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
    model_state_dict = model.state_dict()
    for key in model_state_dict:
        #         pdb.set_trace()
        if 'encoder' in key:
            init.normal_(model_state_dict[key].data)

In [59]:
model_vae.apply(weights_init_rondom)
model_vae.to('cuda')

VAE(
  (encoder): BertForLatentConnectorAVG(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 512, padding_idx=0)
      (position_embeddings): Embedding(512, 512)
      (token_type_embeddings): Embedding(2, 512)
      (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-3): 4 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (LayerNorm): LayerNorm((512,), eps=1e-12, elementw

In [60]:
calc_rec_lgy(model_vae, tokenizer_encoder, tokenizer_decoder,eval_dataloader, "cuda", True, ns=1)

/data/jieqi/.venv/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/data/jieqi/.venv/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/data/jieqi/.venv/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()

{'bleu': 4.939666271115757e-230}